# L&G Scheduler

# Background
In the summer of 2024 I interned at Leaf & Grain, a healthy salad resturant in Houston Texas as a strategic analyist. My final project was to build a scheduling algorithum that would take in numerious inputs such as employee availability, ability and company constraints both financial and operational. To solve such a problem, I applied methods from linear optimization solvers on python. 

Note: the program will not run because token is needed. 

This project was spit up into 3 files with correlated purposes. 
1. Scheduler.py searches for optimal function and outputs data into schedule_data.json file at the location of the program 
2. API_interface.py grabs data from google sheet (this will automatically run with software)
3. API_paster.py pastes schedule_data.json information back into scheduler sheet


### Download neccesary libraries
before initalizing any calls we need to import libraries

In [ ]:
pip install pandas
pip install numpy
pip install matplotlib
pip install pulp


## API Interface File
There will be a large table on google sheets that will hold all information for the scheudling. The Google Sheets could be considered the user interface of a manager so that the program can be usable for those that do not know how to use Python. The Sheet makes variables more dynamic from week to week as roster availability and operation requirements change. 


In [ ]:
"""
Created on Wed Jul 10 16:50:28 2024

@author: williamluik
"""

import os
import pandas as pd

def grab_sheet(file_path):
    """
    Replaces the Google Sheets API call with CSV file reading.
    :param file_path: Path to the CSV file.
    :return: DataFrame containing the CSV data.
    """
    try:
        # Load CSV into a DataFrame
        data = pd.read_csv(file_path)
        return data
    except FileNotFoundError as e:
        print(f"Error: {e}")
        return None
        
        
        employee_names =[]
        employee_pref = {}
        ability ={}
        availability = {} 
        force_shift ={}
        allocated_max_hours ={}
        allocated_min_hours={}
        emp_num_shifts ={}
        start_shift_ranges={}
        emp_shift_len= {}
        
        
        
        def column_to_index(column):
            index = 0
            for char in column:
                index = index * 26 + (ord(char.upper()) - ord('A')) + 1
            return index - 1
        
        def get_cell_value(cell_location ):
            if service is None or SPREADSHEET_ID is None:
                raise ValueError("Service and Spreadsheet ID must be provided")
        
            try:
                cell_result = service.spreadsheets().values().get(
                    spreadsheetId=SPREADSHEET_ID , range=f"Back_end!{cell_location}").execute()
                cell_value = cell_result.get("values", [])[0][0] if cell_result.get("values", []) else None
                return cell_value
            except HttpError as error:
                print(f"An error occurred: {error}")
                return None
        
        def get_range_values(range_location, data_type=str):
            if service is None or SPREADSHEET_ID is None:
                raise ValueError("Service and Spreadsheet ID must be provided")
        
            try:
                range_result = service.spreadsheets().values().get(
                    spreadsheetId=SPREADSHEET_ID, range=f"Back_end!{range_location}"
                ).execute()
                range_values = range_result.get("values", [])[0] if range_result.get("values", []) else []
        
                # Filter out empty values and convert the remaining values to the specified data type
                if data_type == int:
                    range_values = [int(value) for value in range_values if value]
                elif data_type == float:
                    range_values = [float(value) for value in range_values if value]
                else:
                    range_values = [str(value) for value in range_values if value]
        
                return range_values
            except HttpError as error:
                print(f"An error occurred: {error}")
                return []


        
        days_considering = get_range_values("AIR12:AIX12")

        
        def process_day_availability_and_force_shift(day, employee, row, availability, force_shift, col_start, col_end, col_force_start, col_force_end, col_shift_type, days_considering):
            if day in days_considering:
                # Process availability
                if employee not in availability:
                    availability[employee] = {}
        
                day_list = []
                for avail in row[col_start:col_end]:
                    if avail == "0" or avail == "1":
                        day_list.append(int(avail))
                availability[employee][day] = day_list
        
                # Process force shift
                force_list = []
                for force_time in row[col_force_start:col_force_end]:
                    if force_time == "0" or force_time == "1":
                        force_list.append(int(force_time))
        
                if force_list:
                    if employee not in force_shift:
                        force_shift[employee] = {}
                    force_shift[employee][day] = {"segments": force_list}
                    if row[col_shift_type] == "FP":
                        force_shift[employee][day]["job_type"] = 0
                    elif row[col_shift_type] == "BM":
                        force_shift[employee][day]["job_type"] = 1
                    elif row[col_shift_type] == "O":
                        force_shift[employee][day]["job_type"] = 2
                    elif row[col_shift_type] == "HYB":
                        force_shift[employee][day]["job_type"] = 3
                        
        def populate_hourly_requirements_BM(values, end_req_col):
            day_column_map = {
                "Sunday": "AJG",
                "Monday": "AJA",
                "Tuesday": "AJB",
                "Wednesday": "AJC",
                "Thursday": "AJD",
                "Friday": "AJE",
                "Saturday": "AJF"
            }
            
            days_considering_list = ["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]
            hourly_requirements_BM = {day: [] for day in days_considering_list}
            
            if values:
                for row in values[2:end_req_col]:  # Ensure this includes the ending
                    for day in days_considering_list:
                        day_column = day_column_map[day]
                        if len(row) > column_to_index(day_column):
                            day_info = row[column_to_index(day_column)]
                            if day_info is not None:
                                hourly_requirements_BM[day].append(int(day_info))
            
            return hourly_requirements_BM
        
                
        def process_shift_lengths(values, days_considering):
            shift_len_columns = {
                "Sunday": "AJL",
                "Monday": "AJM",
                "Tuesday": "AJN",
                "Wednesday": "AJO",
                "Thursday": "AJP",
                "Friday": "AJQ",
                "Saturday": "AJR"
            }
        
            shift_lengths = {}
        
            def column_to_index(column):
                index = 0
                for char in column:
                    index = index * 26 + (ord(char.upper()) - ord('A')) + 1
                return index - 1
        
            for row in values[4:]:
                employee = row[0] if row[0] else None
                if employee:
                    shift_lengths[employee] = {}
                    for day, column in shift_len_columns.items():
                        if day in days_considering:
                            col_index = column_to_index(column)
                            shift_length = float(row[col_index]) if col_index < len(row) and row[col_index] != '' else None
                            if shift_length is not None:
                                shift_lengths[employee][day] = shift_length
            return shift_lengths


        





        end_roster_col = int(get_cell_value("B3")) +4 +1  
        
        column_index_AJI = column_to_index("AJI")
        column_index_AJJ = column_to_index("AJJ")
        
        shift_len_sunday_col = column_to_index("AJL")
        shift_len_monday_col = column_to_index("AJM")
        shift_len_tuesday_col = column_to_index("AJN")
        shift_len_wednesday_col = column_to_index("AJO")
        shift_len_thursday_col = column_to_index("AJP")
        shift_len_friday_col = column_to_index("AJQ")
        shift_len_saturday_col = column_to_index("AJJR")
        
        if values:
            shift_lengths= process_shift_lengths(values, days_considering)
            for row in values[4: end_roster_col]:
                employee = row[0] if row[0] else None
                pref = float(row[1]) if row[1] else None
                FP_check = row[3] if row[3] else None
                BM_check =row[4] if row[4] else None
                O_check = row[5] if row[5] else None
                allocated_min_hour = float(row[6]) if row[6] else None
                allocated_max_hour= float (row[7]) if row[7] else None
                num_shifts = int(row[2]) if row[2] is not None and row[2] != '' else None
                
                start_start_shift_range = float (row[column_index_AJI]) if column_index_AJI < len(row) and row[column_index_AJI] else None
                end_start_shift_range =float(row[column_index_AJJ]) if column_index_AJJ < len(row) and row[column_index_AJJ] else None
                
                shift_len_sunday = float (row[shift_len_sunday_col]) if shift_len_sunday_col < len(row) and row[shift_len_sunday_col] else None
                shift_len_monday =float (row[shift_len_monday_col]) if shift_len_monday_col < len(row) and row[shift_len_monday_col] else None
                shift_len_tuesday =  float (row[shift_len_tuesday_col]) if shift_len_tuesday_col < len(row) and row[shift_len_tuesday_col] else None
                shift_len_wednesday = float (row[shift_len_wednesday_col]) if shift_len_wednesday_col < len(row) and row[shift_len_wednesday_col] else None
                shift_len_thursday =float (row[shift_len_thursday_col]) if shift_len_thursday_col < len(row) and row[shift_len_thursday_col] else None
                shift_len_friday =float (row[shift_len_friday_col]) if shift_len_friday_col < len(row) and row[shift_len_friday_col] else None
                shift_len_saturday= float (row[shift_len_saturday_col]) if shift_len_saturday_col < len(row) and row[shift_len_saturday_col] else None
                
                
                
                
                # Adding emp info to datasets only if data is not None
                if employee is not None :
                    employee_names.append(employee)
                if  pref is not None:
                    employee_pref[employee] = pref
                    
                if FP_check and BM_check and O_check is not None:
                    if employee not in ability: #initalize the emp info
                        ability[employee] = {"FP": 0, "BM": 0, "O":0}
                    if FP_check == "TRUE":
                        ability[employee]["FP"] = 1
                    if BM_check == "TRUE":
                        ability[employee]["BM"] = 1
                    if O_check =="TRUE":
                        ability[employee]["O"] = 1
                
                if allocated_min_hour is not None:
                    allocated_min_hours[employee] = ( allocated_min_hour)
                if allocated_max_hour is not None:
                    allocated_max_hours[employee] =  (allocated_max_hour)
                if num_shifts is not None:
                    emp_num_shifts[employee] = (num_shifts)
                if start_start_shift_range is not None and end_start_shift_range is not None:
                    start_shift_ranges[employee] = [start_start_shift_range, end_start_shift_range]
                # for shift_len in [shift_len_sunday, shift_len_monday , shift_len_tuesday , shift_len_wednesday , shift_len_thursday , shift_len_friday ,shift_len_saturday]:
                #     emp_shift_len[employee][]
                        
                #_______________________________________________________________________________________________
                #_______________________________________________________________________________________________
                        


                # Example usage within your main code
                days = {
                    "Sunday": {"avail_start": 'I', "avail_end": 'BU', "force_start": 'BV', "force_end": 'EH', "shift_type": 'EI'},
                    "Monday": {"avail_start": 'EJ', "avail_end": 'GV', "force_start": 'GW', "force_end": 'JI', "shift_type": 'JJ'},
                    "Tuesday": {"avail_start": 'JK', "avail_end": 'LW', "force_start": 'LX', "force_end": 'OJ', "shift_type": 'OK'},
                    "Wednesday": {"avail_start": 'OL', "avail_end": 'QX', "force_start": 'QY', "force_end": 'TK', "shift_type": 'TL'},
                    "Thursday": {"avail_start": 'TM', "avail_end": 'VY', "force_start": 'VZ', "force_end": 'YL', "shift_type": 'YM'},
                    "Friday": {"avail_start": 'YN', "avail_end": 'AAZ', "force_start": 'ABA', "force_end": 'ADM', "shift_type": 'ADN'},
                    "Saturday": {"avail_start": 'ADO', "avail_end": 'AGA', "force_start": 'AGB', "force_end": 'AIN', "shift_type": 'AIO'}
                }

                for day, columns in days.items():
                    process_day_availability_and_force_shift(
                        day,
                        employee,
                        row,
                        availability,
                        force_shift,
                        column_to_index(columns["avail_start"]),
                        column_to_index(columns["avail_end"]),
                        column_to_index(columns["force_start"]),
                        column_to_index(columns["force_end"]),
                        column_to_index(columns["shift_type"]),
                        days_considering
                    )


                
                #_______________________________________________________________________________________________
  
            #_______________________________________________________________________________________________
            #_______________________________________________________________________________________________
            #Operations information
            #tab in because we need to look through the rows again:
            
            

            
            min_shift_len_hrs = float(get_cell_value("AIR3"))
            max_shift_len_hrs = float(get_cell_value("AIR4"))
            hrs_until_break = float(get_cell_value("AIR5"))
            break_len = float (get_cell_value("AIR6"))
            start_hour = float (get_cell_value("AIR7"))
            end_hour = float (get_cell_value("AIR8"))
            min_weekly_FP_hrs = float(get_cell_value("AIR11"))
            sheets_time_limit = int(get_cell_value("AIR16"))
            earliest_shift_end = float (get_cell_value("AIR17"))
            latest_shift_start = float(get_cell_value("AIR18") )
            earliest_latest_flag = str(get_cell_value("AIR19"))
            FP_latest_hr = int(get_cell_value("AIR21"))
            latest_FP_flag = str(get_cell_value("AIR22"))
            

            
            
            
            FP_cutoff_hour= float (get_cell_value("AIR9"))

            total_labor_hour_limit = float (get_cell_value("AIR10"))
            
            
            job_type = get_range_values("AIR13:AIT13")

            min_morning_FP_hrs = get_range_values("AIR14:AIX14", float)
            min_daily_FP_hrs = get_range_values("AIR15:AIX15", float)
            max_daily_O_hrs = get_range_values("AIR20:AIX20", float)

            # print("min_daily_FP", min_daily_FP)
            # print("min_morning_FP", min_morning_FP)



            # print("min_shift_len_hrs", min_shift_len_hrs)
            # print("max_shift_len_hrs", max_shift_len_hrs)
            # print("hours until break", hrs_until_break)
            # print("break_len",break_len)
            # print("start_hour", start_hour)
            # # print("end_hour", end_hour)
            # print("FP cutoff_hour", FP_cutoff_hour)
            # print("min_morning_FP", min_morning_FP_hrs)
            
                    
                        
                        
                        
                
            # print("Employees:", employee_names)
            # print("_____________________________________________")
            # print("Employees Preferences:", employee_pref)
            # print("_____________________________________________")
            # print ("Emp ability:", ability)
            # print("_____________________________________________")
            # print("Allocated min hours", allocated_min_hours)
            # print("_____________________________________________")
            # print("Allocated max hours", allocated_max_hours)
            # print("_____________________________________________")
            # print("force shifts", force_shift)
            # print("_____________________________________________")
            # print("Avilability", availability)
            # print (len(availability["Leah Lem"]["Tuesday"]))
 
            # print("_____________________________________________")
            # print (availability["Jose Juarez"]["Sunday"])
            # print("Days considering", days_considering)
            # print("Job Types:", job_type)


            
        #_______________________________________________________________________________________________
        #_______________________________________________________________________________________________
        #Hours BM Requirement information



        
        end_req_col = int(get_cell_value("AJD1")) + 2  
        
        # Assuming values is defined and fetched from somewhere
        hourly_requirements_BM = populate_hourly_requirements_BM(values, end_req_col)



            
        return {
            "hourly_requirements_BM": hourly_requirements_BM,
            "job_type": job_type,
            "days_considering": days_considering,
            "total_labor_hour_limit": total_labor_hour_limit,
            "min_morning_FP_hrs": min_morning_FP_hrs,
            "FP_cutoff_hour": FP_cutoff_hour,
            "end_hour": end_hour,
            "start_hour": start_hour,
            "break_len": break_len,
            "hrs_until_break": hrs_until_break,
            "max_shift_len_hrs": max_shift_len_hrs,
            "min_shift_len_hrs": min_shift_len_hrs,
            "availability": availability,
            "force_shift": force_shift,
            "allocated_max_hours": allocated_max_hours,
            "allocated_min_hours": allocated_min_hours,
            "ability": ability,
            "employee_pref": employee_pref,
            "employee_names": employee_names,
            "min_daily_FP_hrs": min_daily_FP_hrs, 
            "min_weekly_FP_hrs": min_weekly_FP_hrs, 
            "sheets_time_limit": sheets_time_limit,
            "latest_shift_start": latest_shift_start,
            "earliest_shift_end": earliest_shift_end, 
            "earliest_latest_flag": earliest_latest_flag, 
            "max_daily_O_hrs": max_daily_O_hrs, 
            "FP_latest_hr": FP_latest_hr, 
            "latest_FP_flag": latest_FP_flag, 
            "emp_num_shifts": emp_num_shifts,
            "start_shift_ranges": start_shift_ranges, 
            "shift_lengths": shift_lengths
            
            
        }

     
        
    except HttpError as error:
        print(error)
        


## Scheduler.py file
From a high level, this program will input a large 5-dimensional matrix that represents the availability and ability of each employee for each 15 minute increment of each day of the week. 

### Objective Function
The program has an objective function that as a value that is wanted to maximize or minimize. In our case, minimize total hours worked. 

prob += lp.lpSum(x[i][d][j][a] for i in range(num_employees) for d in range(num_days) for j in range(num_segments) for a in range(num_job))


### Decision Variables
The decision variables are the values within the matrix that will be manipulated to drive us closer to our objective function. In this case, nearly all of the decision variables are bianry. For example, the dinary decision variable x will represent 1 if the employee is working for the time segement and 0 if they are not working. 


### Constraints
The program has many constrains so the final schedule would fit the managerial wants and needs. As we include more constrains, we cut down to large number of possible outcomes thus increasing efficiency; however, with too many constrains we run the risk of making the problem set infesible (this would mean the matrix is indeterminent). So of the constraints include:
1. Ability - prevent employees w/o FP skill to be FP
2. Availability - employees can only work hrs available
3. Single job per hr - prevents double staffing
4. Within allotted shift length (4 - 10 hrs)
5. Can only work 1 shift a day 
6. No breaks - this is only in scheduler; managers will allot breaks 
7. Start time - fixes edge case of employees starting at 6 am 
8. Total labor hours - goal to hold us under labor budget, but objective function is to minimize hrs so keep this at infinity
9. Force shift
10. Hourly BM requirements 
11. Morning FP hrs requirements
12. Employees within allocated hrs range
13. Daily FP hrs requirements 
14. Weekly FP hrs requirements 
15. Ovens 
16. No FP are working past specified time. (optional constraint)





In [ ]:
"""
Created on Mon Jul 15 07:53:18 2024

@author: williamluik
"""


import json
import pulp as lp
from API_interface_final import grab_sheet
import datetime
import os
from math import floor

def lp_solver():
    def print_with_timestamp(message):
        # Get the current date and time
        current_time = datetime.datetime.now()
        # Format the time as a string
        timestamp = current_time.strftime("%Y-%m-%d %H:%M:%S")
        # Print the message with the timestamp
        print(f"[{timestamp}] {message}")
    
    
    
    file_path = "Back_end.csv"
    data = grab_sheet(file_path)
    
    hourly_requirements_BM = data["hourly_requirements_BM"]
    job_type = data["job_type"]
    days_considering = data["days_considering"]
    total_labor_hour_limit = data["total_labor_hour_limit"]
    min_morning_FP_hrs = data["min_morning_FP_hrs"]
    FP_cutoff_hour = data["FP_cutoff_hour"]
    end_hour = data["end_hour"]
    start_hour = data["start_hour"]
    break_len = data["break_len"]
    hrs_until_break = data["hrs_until_break"]
    max_shift_len_hrs = data["max_shift_len_hrs"]
    min_shift_len_hrs = data["min_shift_len_hrs"]
    availability = data["availability"]
    force_shift = data["force_shift"]
    allocated_max_hours = data["allocated_max_hours"]
    allocated_min_hours = data["allocated_min_hours"]
    ability = data["ability"]
    employee_pref = data["employee_pref"]
    employee_names = data["employee_names"]
    min_daily_FP_hrs = data["min_daily_FP_hrs"]
    min_weekly_FP_hrs = data["min_weekly_FP_hrs"]
    sheets_time_limit = data["sheets_time_limit"]
    earliest_shift_end_hrs =data["earliest_shift_end"]
    latest_shift_start_hrs= data["latest_shift_start"]
    earliest_latest_flag = data["earliest_latest_flag"]
    max_daily_O_hrs = data["max_daily_O_hrs"]
    FP_latest_hr = data["FP_latest_hr"]
    latest_FP_flag= data["latest_FP_flag"]
    emp_num_shifts = data["emp_num_shifts"]
    start_shift_ranges = data["start_shift_ranges"]
    shift_lengths = data["shift_lengths"]
    
    
    
    print("_______Operations info check________")
    print("days_considering", days_considering)
    print("total_labor_hour_limit", total_labor_hour_limit)
    print("min_morning_FP_hrs______________", min_morning_FP_hrs)
    print("FP_cutoff_hour",FP_cutoff_hour, " AM")
    
    # Define data for hours. based on 24 hour time
    
    print("__________________________________________________")
    print("Run optimization software")
    print("__________________________________________________")
    print_with_timestamp("start time of run")
    print( "Software will run for", sheets_time_limit/60, " minutes" )

    

    
    end_hour_backend =22
    start_hour_backend = 6
    
    num_hours = end_hour_backend - start_hour_backend   # Number of hours in the specified range
    segment_minutes =15
    num_segments = int(((num_hours) * (60 / segment_minutes)))  # Number of 15-minute segments. 
    end_hour_backend_segs = int (end_hour_backend * (60 / segment_minutes))
    
    #change ealiest and latest shift star/end to 15-min segments
    earliest_shift_end = int((earliest_shift_end_hrs- start_hour_backend)  * (60 / segment_minutes))
    latest_shift_start= int((latest_shift_start_hrs- start_hour_backend)  * (60 / segment_minutes))
    
    #segment the shift lengths
    def convert_shift_lengths_to_segments(shift_lengths, segment_minutes):
        shift_lengths_segs = {}
        factor = 60 / segment_minutes
    
        for employee, days in shift_lengths.items():
            shift_lengths_segs[employee] = {}
            for day, length in days.items():
                shift_lengths_segs[employee][day] = int(length * factor)
    
        return shift_lengths_segs
    shift_lengths_segs = convert_shift_lengths_to_segments(shift_lengths, segment_minutes)


    


    # Create min_morning_FP by multiplying each element in min_morning_FP_hrs by (60 / segment_minutes)
    min_morning_FP = [hours * (60 / segment_minutes) for hours in min_morning_FP_hrs]
    
    #change the start range listing into proper segments
    start_shift_ranges_segs = {
        employee: [
            int((start_time - start_hour_backend) * (60 // segment_minutes)),
           int(1+ (end_time - start_hour_backend) * (60 // segment_minutes))
        ]
        for employee, (start_time, end_time) in start_shift_ranges.items()
        }



    
    
    #change FP latest hr to the 15 min segment on the j range. must subtract from start hr backend
    FP_latest = int((FP_latest_hr- start_hour_backend)  * (60 / segment_minutes))

    #change Hrs until break into 15 min segments 
    segs_until_break = int( hrs_until_break * (60 / segment_minutes))
    
    num_days = len(days_considering)  

    num_job = len(job_type) + 1 # add one to include job types as they are not grabbed from the google sheet
    
    # min max shift hours
    min_shift_len = min_shift_len_hrs *(60/segment_minutes)
    max_shift_len = max_shift_len_hrs *(60/segment_minutes)
    
    # Calculate min_daily_FP_hrs by multiplying min_daily_FP by (60 / segment_minutes)
    min_daily_FP = [hours * (60 / segment_minutes) for hours in min_daily_FP_hrs]
    min_weekly_FP = min_weekly_FP_hrs * (60 / segment_minutes)
    
    #calculate min daily oven hours into segments 
    max_daily_O = [int(hours * (60 / segment_minutes)) for hours in max_daily_O_hrs]
    #names
    num_employees = len(employee_names)
    
    
    
    # Calculate the indices for hours before the cutoff hour
    hours_before_cutoff = [j for j in range(int(( FP_cutoff_hour - start_hour_backend ) * (60/segment_minutes)))] # plus 1 so that we include the cutooff hour. Range is not inclusive
    
    
    # Initialize the LP problem
    prob = lp.LpProblem("Employee_Scheduling", lp.LpMinimize)
    
    # Decision variables
    # x[i][d][j] is 1 if employee i works during hour j on day d, 0 otherwise
    x = [[[[lp.LpVariable(f"x_{i}_{d}_{j}_{a}", cat=lp.LpBinary) for a in range(num_job)] for j in range(num_segments)] for d in range(num_days)] for i in range(num_employees)]
    
    # y[i][d] is 1 if employee i works any hours at all on day d, 0 otherwise
    y = [[lp.LpVariable(f"y_{i}_{d}", cat=lp.LpBinary) for d in range(num_days)] for i in range(num_employees)]
    
    # f[i][d][j] is 1 for the one hour when they start to work on day d. This is the start flag so only consecutive shifts
    f = [[[lp.LpVariable(f"f_{i}_{d}_{j}", cat=lp.LpBinary) for j in range(num_segments)] for d in range(num_days)] for i in range(num_employees)]
    
    #bk[i][d] is 1 if someone is granted a break for that day. Only granted if shift len > hrs until break
    bk = [[lp.LpVariable(f"bk_{i}_{d}", cat=lp.LpBinary) for d in range(num_days)] for i in range(num_employees)]
    
    floor_segments = [[lp.LpVariable(f"floor_segments_{i}_{d}", cat=lp.LpInteger) for d in range(num_days)] for i in range(num_employees)]
    
    

    # Objective function: minimize the hours worked
    prob += lp.lpSum(x[i][d][j][a] for i in range(num_employees) for d in range(num_days) for j in range(num_segments) for a in range(num_job))
    

    
    
    #Constraints:
    for i in range(num_employees):
        # Constraint: total hours worked over the days must be under the maximum allowed hours per employee
        prob += lp.lpSum(x[i][d][j][a] for d in range(num_days) for j in range(num_segments) for a in range(num_job)) <=  allocated_max_hours [employee_names[i]] *(int (60/segment_minutes))
        prob += lp.lpSum(x[i][d][j][a] for d in range(num_days) for j in range(num_segments) for a in range(num_job)) >=  allocated_min_hours [employee_names[i]] *(int(60/segment_minutes))
        for d in range(num_days):
            #Constraint: Min & Max hours per emploee per day or none hours at all
            prob += lp.lpSum(x[i][d][j][a] for j in range(num_segments) for a in range(num_job)) >= min_shift_len * y[i][d]  # Minimum hours if y[i][d] == 1
            prob += lp.lpSum(x[i][d][j][a] for j in range(num_segments) for a in range(num_job)) <= max_shift_len * y[i][d]  # Maximum hours if y[i][d] == 1
            
            # ensure starting shift properly accounted for for edge effects
            prob += f[i][d][0] == lp.lpSum(x[i][d][0][a] for a in range(num_job))
            
            # Constraint to calculate the number of segments worked
            prob += floor_segments[i][d] == lp.lpSum(x[i][d][j][a] for j in range(num_segments) for a in range(num_job - 1))  # Exclude breaks
            
            # Constraint: Allocate breaks only if segments worked > 24
            prob += floor_segments[i][d] >= 25 * bk[i][d]
            prob += floor_segments[i][d] <= 25 * bk[i][d] + 24  # Ensure break only if segments are > 24
            
            # Constraint: Ensure exactly 2 break segments if bk[i][d] == 1
            prob += lp.lpSum(x[i][d][j][num_job-1] for j in range(num_segments)) == 2 * bk[i][d]
    
            for j in range(num_segments - 1):
                # Sum over all job types to check if there's a transition from 0 to 1 in total working hours
                prob += f[i][d][j+1] >= lp.lpSum(x[i][d][j+1][a] for a in range(num_job)) - lp.lpSum(x[i][d][j][a] for a in range(num_job))
            
            # Constraints: ensure f[i][d][j] sums to 1 for each employee per day. Can only start once
            prob += lp.lpSum(f[i][d][j] for j in range(num_segments)) == y[i][d]
            for j in range(num_segments) : 
                prob += lp.lpSum(x[i][d][j][a] for a in range(num_job)) <=1  #Constraint: can only work one shift type at a time
                
                #Constraint: Consider skill abilities If employee does not have FP ability, they cannot be assigned to FP tasks (job_type[0])
                if ability[employee_names[i]]["FP"] == 0:
                    prob += x[i][d][j][0] == 0
                # If employee does not have BM ability, they cannot be assigned to BM tasks (job_type[1])
                if ability[employee_names[i]]["BM"] == 0:
                    prob += x[i][d][j][1] == 0
                # DO not consider ppl for ovens if they are not available to do it
                if ability[employee_names[i]]["O"] == 0:
                    prob += x[i][d][j][2] == 0
                
                #Constraint: emp can only work during available hours
                if availability[employee_names[i]][days_considering[d]][j] == 0:  
                    for a in range(num_job):
                        prob += x[i][d][j][a] == 0
            #Constraint to make sure that food preps are not working past a certain time at Allen most specifically 
            if latest_FP_flag =="TRUE":
                if ability[employee_names[i]]["BM"] ==0: #only constrain the people whos are soly Food preps. For Allen this is only 2 specific people we dont want working late. AKA the good food prepers
                    for j in range(FP_latest, num_segments ):
                        for a in range(num_job-1):
                            prob += x[i][d][j][a] ==0
    

    # Constraint for forcing some people to only work a specified number of days
    for i in range(num_employees):
        employee_name = employee_names[i]  # Assuming you have a list of employee names
        if employee_name in emp_num_shifts:
            prob += lp.lpSum(y[i][d] for d in range(num_days)) == emp_num_shifts[employee_name]

    #COnstraint: make some people stat in a specified range
    for i in range(num_employees):
        employee_name = employee_names[i]
        for d in range(num_days):
            if employee_name in start_shift_ranges_segs:
                #so I am making it so that the start time will be within this range if they are working. if not working then y ==0 and this start time has to be 0 for the range
                prob += lp.lpSum(f[i][d][j] for j in range (start_shift_ranges_segs[employee_name][0], start_shift_ranges_segs[employee_name][1] )) == y[i][d]  
                # prob += lp.lpSum(x[i][d][j][a] for j in range(start_shift_ranges_segs[employee_name][0], start_shift_ranges_segs[employee_name][1]) for a in range(num_job)) >= y[i][d]




    if earliest_latest_flag == "TRUE":
        for i in range (num_employees):
                for d in range(num_days):
                    prob += lp.lpSum(x[i][d][j][a] for j in range(latest_shift_start, earliest_shift_end) for a in range(num_job)) >= y[i][d] * (earliest_shift_end - latest_shift_start)
       
                 
    
    for d in range(num_days):
        #Constraint: have more than the min # of FP hrs and O per day
        prob += lp.lpSum(x[i][d][j][0] for j in range(num_segments) for i in range(num_employees)) >= min_daily_FP[d]
        prob += lp.lpSum(x[i][d][j][2] for j in range(num_segments) for i in range(num_employees)) == max_daily_O[d]
        # Constraint: Ensure at least a certain number of hours are worked for FP before the cutoff hour
        prob += lp.lpSum(x[i][d][j][0] for i in range(num_employees) for j in hours_before_cutoff) >= min_morning_FP[d]
                         
        for j in range(num_segments):
            # Constraint: Ensure more than the specified number of people are working BM each hour
            prob += lp.lpSum(x[i][d][j][1] for i in range(num_employees)) >=  hourly_requirements_BM[days_considering [d]][j]
            #Constraint that only 1 person can be working at the Oven at a time
            prob +=lp.lpSum(x[i][d][j][2] for i in range(num_employees)) <= 1
        
    
    

    #Constraint: be above the min weekly FP hours
    prob += lp.lpSum(x[i][d][j][0] for i in range(num_employees) for j in range(num_segments) for d in range(num_days)) >= min_weekly_FP
                   
    # Constraint: total hours worked over the days must be under the total labor hour limit
    prob += lp.lpSum(x[i][d][j][a] for i in range(num_employees) for d in range(num_days) for j in range(num_segments) for a in range(num_job)) <=  total_labor_hour_limit * (60/segment_minutes)
    
    
    
    # Constraint: Force people in the "force_shift" to be a part of said shift for a specific job type 'a'
    for employee, shifts in force_shift.items():
        i = employee_names.index(employee)
        for day, details in shifts.items():
            d = days_considering.index(day)
            segments = details["segments"]
            job_type = details["job_type"]
            if job_type in [0, 1, 2]:  # This means they are forced to specifically be a BM, FP, or O
                for j, forced in enumerate(segments):
                    if forced == 1:
                        prob += x[i][d][j][job_type] == 1  # Force employee to work job type 'a' during specified segments
            elif job_type == 3:  # This means they can be either BM or FP
                for j, forced in enumerate(segments):
                    if forced == 1:
                        prob += lp.lpSum([x[i][d][j][a] for a in range(2)]) == 1  # Force employee to work either BM or FP


    # Constraint: Ensure total hours worked per day for each employee matches the shift_lengths_segs value
    for i in range(num_employees):
        employee_name = employee_names[i]
        if employee_name in shift_lengths_segs:
            for d in range(num_days):
                if days_considering[d] in shift_lengths_segs[employee_name]:
                    shift_length = shift_lengths_segs[employee_name][days_considering[d]]
                    prob += lp.lpSum(x[i][d][j][a] for j in range(num_segments) for a in range(num_job)) == shift_length
                    
        
    
    
    
    # Set a time limit in seconds 
    time_limit_seconds = sheets_time_limit
    
    # Solve the LP problem with CBC and a time limit
    status = prob.solve(lp.PULP_CBC_CMD(timeLimit=time_limit_seconds))
    
    
    
    
    # Output results
    print("Status:", lp.LpStatus[prob.status])
    return status, employee_names, days_considering, num_employees, num_days, num_segments, num_job, start_hour, segment_minutes, y, x, f, employee_pref
    

def get_lp_solver_outputs():
    return lp_solver()

    
    
def prepare_schedule_data(employee_names, days_considering, num_employees, num_days, num_segments, num_job, start_hour, segment_minutes, y, x, f, employee_pref):
    data = []
    headers = ["Employee", "Day", "Start Time", "Job Type", "Total Hours"]
    data.append(headers)

    total_team_hours = 0
    day_off_count = 0
    start_hour_index=6

    for i in range(num_employees):
        total_weekly_hours = 0
        for d in range(num_days):
            total_day_hours = 0
            if lp.value(y[i][d]) == 1:
                for j in range(num_segments):
                    for a in range(num_job):
                        if lp.value(x[i][d][j][a]) == 1:
                            total_day_hours += 1
                            total_weekly_hours += 1
                            total_team_hours += 1

                            job_type = "FP" if a == 0 else "BM" if a == 1 else "O" if a ==2 else "BK" if a == 3 else "Unknown"
                            hour = int(start_hour_index + (j * segment_minutes) // 60)
                            minute = int((j * segment_minutes) % 60)
                            time_str = f"{hour:02d}:{minute:02d}"
                            data.append([employee_names[i], days_considering[d], time_str, job_type, total_day_hours / (60/segment_minutes)])
            else:
                data.append([employee_names[i], days_considering[d], "Off", "", 0])

        total_weekly_hours = total_weekly_hours / (60/segment_minutes)
        data.append([employee_names[i], "Total Hours for the week", "", "", total_weekly_hours])

    total_team_hours = total_team_hours / (60/segment_minutes)
    summary = [
        ["Total team hours this week", total_team_hours],
        ["Total number of people who had day off and were available", day_off_count]
    ]
    data.extend(summary)
    return data

def save_data_to_json(data, filename):
    with open(filename, 'w') as f:
        json.dump(data, f)

def scheduler(status, employee_names, days_considering, num_employees, num_days, num_segments, num_job, start_hour, segment_minutes, y, x, f, employee_pref):
    if status != lp.LpStatusOptimal:
        print("Optimization did not find an optimal solution.")
        return

    data = prepare_schedule_data(
        employee_names, days_considering, num_employees, num_days, 
        num_segments, num_job, start_hour, segment_minutes, y, x, f, employee_pref
    )
    save_data_to_json(data, 'schedule_data.json')

def run_scheduler():
    status, employee_names, days_considering, num_employees, num_days, num_segments, num_job, start_hour, segment_minutes, y, x, f, employee_pref = get_lp_solver_outputs()
    scheduler(status, employee_names, days_considering, num_employees, num_days, num_segments, num_job, start_hour, segment_minutes, y, x, f, employee_pref)




ModuleNotFoundError: No module named 'API_interface_final'

### API Paster file 

This file will take the matrix of the linear optimization solver and interpret it back to be pasted into the Google Sheets so that managers can read a formatted schedule for the next week. 

In [ ]:
"""
Created on Wed Jul 10 09:09:51 2024

@author: williamluik
"""

import json
import os
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from Scheduler_final import run_scheduler
from API_interface_final import grab_sheet


SCOPES = ["https://www.googleapis.com/auth/spreadsheets"]
SPREADSHEET_ID = "110KkeoBMKEB48OogS5E5X3LZN4liKbKEQJzpx8sYv8E"
SHEET_NAME = "Final_schedule"

def get_service():
    creds = None
    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                "scheduler_credentials.json", SCOPES)
            creds = flow.run_local_server(port=0)
        with open("token.json", "w") as token:
            token.write(creds.to_json())
    service = build("sheets", "v4", credentials=creds)
    return service

def load_data_from_json(filename):
    with open(filename, 'r') as f:
        data = json.load(f)
    return data

def update_sheet(service, spreadsheet_id, sheet_name, data):
    try:
        body = {
            "range": f"{sheet_name}!A1",
            "majorDimension": "ROWS",
            "values": data
        }
        result = service.spreadsheets().values().update(
            spreadsheetId=spreadsheet_id, range=f"{sheet_name}!A1",
            valueInputOption="RAW", body=body).execute()
        print(f"{result.get('updatedCells')} cells updated.")
    except HttpError as error:
        print(f"An error occurred: {error}")

def main():
    run_scheduler()
    service = get_service()
    data = load_data_from_json('schedule_data.json')
    update_sheet(service, SPREADSHEET_ID, SHEET_NAME, data)

if __name__ == "__main__":
    main()


ModuleNotFoundError: No module named 'Scheduler_final'

## Additional Files
In order to make the program as usable as possible for the resturant managers, I created subfiles that would be called to import the neccessary libraries and run the code. The ensured that others would not need to get an IDE to run the code seamlessly. 

### Lib Requirements.txt

In [ ]:
google-auth>=1.0.0
google-auth-oauthlib==0.4.6
google-auth-httplib2==0.1.0
google-api-python-client==2.0.2
pulp==2.4

### Run.sh

In [ ]:
#!/bin/bash

# Create a virtual environment
python3 -m venv venv

# Activate the virtual environment
source venv/bin/activate

# Install the required dependencies
pip install -r lib_requirements.txt

# Run the main Python script
python main.py

# Deactivate the virtual environment
deactivate



## Conclusion
In the end, this code will product usable results when the constrains are not too tightly bound. There are efficency issue when using so many for loops in the lp solver. There are cyber security vulnerabilities that would need to be fortified if a company would want to implement their code into a larger database in protected information. 

To imporve the code I would improve the robustness by not just saying the answer is infesible but stating which constraint is being too restrictive. 


Thank you chatGPT for providing direction. 